# NoteBook utilizado para criação e avaliação dos Hiperparâmetros da árvore de decisão.

# Será inicialmente carregado com o DataSet definido no Notebook "TCC_TYAGO_AnaliseAtributos.ipynb".

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
from sklearn import datasets, tree
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import LabelEncoder
from scipy.stats import randint
from sklearn.ensemble import GradientBoostingClassifier

Carregamento do arquivo e faz os seguintes tratamento de dados:


*   Remove registros Nulos
*   Remove regitros Inválidos
*   Substitui valores nulos por zero
*   Cria campo de diferança.   

In [ ]:
pd_dados = pd.read_excel("/content/sample_data/TCC_Dados_Final1_ni.xlsx", sheet_name=0)     
#pd_dados = pd.read_csv(arquivo, sheet_name=0) 

print(pd_dados.describe())
        
print("\nRemovendo registros nulos do arquivo")
# dropna remove os registros faltantes    
pd_dados.dropna(axis=0, subset=['sit_cadastral'], inplace = True) 
pd_dados.dropna(axis=0, subset=['cnae3'], inplace = True)
pd_dados.dropna(axis=0, subset=['ni'], inplace = True)


#Remove registros que tem CNAE Inválido ou Não Informado
pd_dados['sit_cadastral'] = pd_dados['sit_cadastral'].astype('str')
pd_dados = pd_dados[~pd_dados['sit_cadastral'].str.contains('NULA')]

print("\nRemovendo registros inválidos do arquivo")
#Remove registros que tem CNAE Inválido ou Não Informado
pd_dados['cnae3'] = pd_dados['cnae3'].astype('str')
pd_dados = pd_dados[~pd_dados['cnae3'].str.contains('IN')]
pd_dados = pd_dados[~pd_dados['cnae3'].str.contains('NI')]

print("\nSubstitui valores nulos por Zero para os campos de Débitos")
#Coloca zero quando não tem valor, para não dá erro no calculo da diferença. Quando está nulo é por que não tem mais débito, ou seja, está zerado
pd_dados[['valorm1']] = pd_dados[['valorm1']].fillna(value = 0)
pd_dados[['valorm3']] = pd_dados[['valorm3']].fillna(value = 0)

print("\nCriando campos de comparação")
#calcula os campos de comparação para saber se o débito aumentou o diminuiu
pd_dados["Dif1"] = pd_dados.valorm1 - pd_dados.valor
pd_dados["Dif3"] = pd_dados.valorm3 - pd_dados.valor

print(pd_dados.describe())

print("\nSalvando o arquivo: {0}".format("/content/sample_data/TCC_Dados_Final1_ni_Formatado.xlsx"))
pd_dados.to_excel("/content/sample_data/TCC_Dados_Final1_ni_Formatado.xlsx", index=False)

                  ni   nat_juridica         valor       valorm1       valorm3
count  389285.000000  389285.000000  3.892850e+05  3.695480e+05  3.556940e+05
mean   191398.469052     218.066075  4.008667e+04  5.101589e+04  4.939816e+04
std    111414.146668      30.856279  3.546057e+06  4.138557e+06  2.927664e+06
min         0.000000     102.000000  1.000000e-02  1.000000e-02  1.000000e-02
25%     94644.000000     213.000000  1.961300e+02  2.005375e+02  2.035800e+02
50%    190997.000000     213.000000  6.393700e+02  6.492100e+02  6.606050e+02
75%    287930.000000     213.000000  1.837220e+03  1.863757e+03  1.845945e+03
max    384661.000000     401.000000  1.889909e+09  1.910075e+09  9.480777e+08

Removendo registros nulos do arquivo

Removendo registros inválidos do arquivo


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Substitui valores nulos por Zero para os campos de Débitos

Criando campos de comparação


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

                  ni   nat_juridica         valor       valorm1       valorm3  \
count  389128.000000  389128.000000  3.891280e+05  3.891280e+05  3.891280e+05   
mean   191394.907732     218.067859  4.008731e+04  4.843335e+04  4.513832e+04   
std    111410.521081      30.859295  3.546760e+06  4.033096e+06  2.799085e+06   
min         0.000000     102.000000  1.000000e-02  0.000000e+00  0.000000e+00   
25%     94636.750000     213.000000  1.961300e+02  1.382700e+02  1.226300e+02   
50%    191005.500000     213.000000  6.393700e+02  6.053800e+02  5.649450e+02   
75%    287920.250000     213.000000  1.837510e+03  1.742180e+03  1.654602e+03   
max    384661.000000     401.000000  1.889909e+09  1.910075e+09  9.480777e+08   

               Dif1          Dif3  
count  3.891280e+05  3.891280e+05  
mean   8.346039e+03  5.051010e+03  
std    3.943316e+06  2.695450e+06  
min   -9.439711e+08 -9.418316e+08  
25%    1.400000e-01  2.400000e-01  
50%    6.900000e-01  1.790000e+00  
75%    2.240000e+0

Função que define se Aumentou, Diminuiu ou Manteve

In [ ]:
def encode_units(x):
  if x < 0:
    return "Diminuiu"
  elif x > 2.5:
    return "Aumentou"
  else:
      return "Manteve"

Campos do Modelo a ser treinado

    0 - ni	
    1 - classe	
    2 - sit_cadastral	
    3 - diferenciada	
    4 - simei	
    5 - simples	
    6 - porte	
    7 - nat_juridica	
    8 - cnae3	
    9 - valor
    10 - valorm1	
    11 - valorm3	
    12 - Dif1
    13 - Dif3

Treinamendo de Modelo sem Particionar a árvore e com Hiperparâmetros pre-definidos, "GRID SEARCH"

In [ ]:
pd_debitos = pd.read_excel("/content/sample_data/TCC_Dados_Final1_ni_Formatado.xlsx", sheet_name=0) 
print("\nDimensões: {0}".format(pd_debitos.shape))
#print("\nCampos: {0}".format(pd_debitos.keys()))
#print(pd_debitos.describe(), sep='\n')
print("\nQuantidade de Registros antes: {0}".format(pd_debitos["classe"].count()))

#Tranforma os valores de diferença em -1 se diminuiu e 1 se aumentou o débito
pd_debitos["Dif1"] = pd_debitos["Dif1"].apply(encode_units)
pd_debitos["Dif3"] = pd_debitos["Dif3"].apply(encode_units)
#apaga os valores sem diferença, manteve o débito igual
pd_debitos.dropna(axis=0, subset=['Dif1'], inplace = True)
pd_debitos.dropna(axis=0, subset=['Dif3'], inplace = True)

#print("\nQuantidade de Registros depois: {0}".format(pd_debitos["classe"].count()))

var_mod =['classe','sit_cadastral', 'diferenciada', 'simei','simples', 'porte', 'nat_juridica', 'cnae3'] 
le = LabelEncoder() 
print(pd_debitos.keys())
for i in var_mod: 
    pd_debitos[i] = le.fit_transform(pd_debitos[i]) 
    print(pd_debitos[i].sort_values().unique().tolist())
    print(le.inverse_transform(pd_debitos[i].sort_values().unique().tolist()))


#define a coluna de resultado
#print(pd_debitos.iloc[:,13:14])
y_dados_m1 = le.fit_transform(pd_debitos.iloc[:,12:13])
y_dados_m3 = le.fit_transform(pd_debitos.iloc[:,13:14])
print(le.inverse_transform([0,1,2]))
#print(y_train)    

X_dados = pd_debitos.iloc[:,0:9]

for criterios_decisao in [['gini', 1, 2],['gini',50, 100],['gini', 50, 200], ['gini', 50, 500], ['gini', 50, 1000], ['gini', 100, 500], ['gini', 100, 1000], ['gini', 200, 500], ['entropy', 1, 2],['entropy',50, 100],['entropy', 50, 200], ['entropy', 50, 500], ['entropy', 50, 1000], ['entropy', 100, 500], ['entropy', 100, 1000], ['entropy', 200, 500]]:
    #Cria o modelo com os HyperParametros configurados, nesse caso utiliza os Defaults
    
    debitos_tree = DecisionTreeClassifier(random_state=0, criterion=criterios_decisao[0], min_samples_leaf=criterios_decisao[1], min_samples_split=criterios_decisao[2])
    #debitos_tree = DecisionTreeClassifier(random_state=0)
    #print("DecisionTreeClassifier")
    debitos_tree = debitos_tree.fit(X_dados, y_dados_m1) #comando que irá treinar é o FIT
    #print("Acurácia", debitos_tree.score(X_train, y_train)) #método Scrore calcula a Acurária da base de Treinamento

    Train_predict = debitos_tree.predict(X_dados) 
    #print("Acurácia (previsão):", accuracy_score(y_dados_m1, Train_predict)) 
    #print(classification_report(y_dados_m1, Train_predict, target_names=["Aumentou", "Diminuiu", "Manteve"]))

    print("Acurancia M1: {0}, Acurancia M3: {1}, criterion: {2}, min_samples_leaf: {3}, min_samples_split: {4}".format(accuracy_score(y_dados_m1, Train_predict), accuracy_score(y_dados_m3, Train_predict), debitos_tree.criterion, debitos_tree.min_samples_leaf, debitos_tree.min_samples_split))
    
    Train_predict = debitos_tree.predict(X_dados) 
    #print("Acurácia (previsão M3):", accuracy_score(y_dados_m3, Train_predict)) 
    print(classification_report(y_dados_m3, Train_predict, target_names=["Aumentou", "Diminuiu", "Manteve"]))

    #Matriz de Confusão
    cnf_matrix = confusion_matrix(y_dados_m3, Train_predict)
    cnf_table = pd.DataFrame(data=cnf_matrix, index=["Aumentou", "Diminuiu", "Manteve"], columns=["Aumentou (prev)", "Diminuiu (prev)", "Manteve (prev)"])
    print(cnf_table)


Dimensões: (389128, 14)

Quantidade de Registros antes: 389128
Index(['ni', 'classe', 'sit_cadastral', 'diferenciada', 'simei', 'simples',
       'porte', 'nat_juridica', 'cnae3', 'valor', 'valorm1', 'valorm3', 'Dif1',
       'Dif3'],
      dtype='object')
[0, 1]
['NPREV' 'PREV']
[0, 1, 2, 3]
['ATIVA' 'BAIXADA' 'INAPTA' 'SUSPENSA']
[0, 1]
['NÃO' 'SIM']
[0, 1]
['NÃO' 'SIM']
[0, 1]
['NÃO' 'SIM']
[0, 1, 2, 3]
['PJ-D' 'PJ-DEMAIS' 'PJ-E' 'PJ-M']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44]
[102 103 105 106 108 110 111 112 115 117 118 121 122 124 127 133 201 203
 204 205 206 207 213 214 215 217 223 224 225 230 231 232 303 306 307 308
 311 313 320 322 326 327 330 399 401]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Acurancia M1: 1.0, Acurancia M3: 0.6744361752431076, criterion: gini, min_samples_leaf: 1, min_samples_split: 2
              precision    recall  f1-score   support

    Aumentou       0.81      0.44      0.57    166789
    Diminuiu       0.89      0.49      0.64     64597
     Manteve       0.60      1.00      0.75    157742

    accuracy                           0.67    389128
   macro avg       0.77      0.64      0.65    389128
weighted avg       0.74      0.67      0.65    389128

          Aumentou (prev)  Diminuiu (prev)  Manteve (prev)
Aumentou            73508             3687           89594
Diminuiu            16437            31840           16320
Manteve               567               81          157094
Acurancia M1: 0.7087025348985424, Acurancia M3: 0.4642456980736416, criterion: gini, min_samples_leaf: 50, min_samples_split: 100
              precision    recall  f1-score   support

    Aumentou       0.50      0.18      0.26    166789
    Diminuiu       0.52      0.0

# Nova seção

# Nova seção